# Circle Footprint Filter Demo

A demonstration validating the PPCircleFootprint filter.

In [ ]:
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord

In [ ]:
from sorcha.modules.PPApplyFOVFilter import PPCircleFootprint, PPGetSeparation

We begin by loading in a set of randomised artificial observations which were all generated to lie on the same field, within 2.1 degrees of field centre.

In [ ]:
def get_sql_data(database, rows_start, nrows):
    
    con = sql.connect(database)
    observations = pd.read_sql("""SELECT * FROM observations LIMIT """ + str(rows_start) + ',' + str(nrows), con)
    
    return observations

In [ ]:
def get_separation(obj_RA, obj_Dec, cen_RA, cen_Dec):
    
    obj_coord = SkyCoord(ra=obj_RA, dec=obj_Dec, unit="deg")
    cen_coord = SkyCoord(ra=cen_RA, dec=cen_Dec, unit="deg")

    sep = obj_coord.separation(cen_coord)

    return sep.degree

In [ ]:
db_path = "oneline_v2.0.db"
LSSTdf = get_sql_data(db_path, 0,1)

In [ ]:
LSSTdf

In [ ]:
dfobs = pd.read_csv("footprintFilterValidationObservations.csv", delim_whitespace=True)

In [ ]:
dfobs = pd.merge(dfobs, LSSTdf, left_on="FieldID", right_on="observationId", how="left")

In [ ]:
dfobs

We can roughly plot the distance from the field centre in RA and Dec.

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))
ax.scatter(dfobs['fieldRA']-dfobs['AstRA(deg)'],dfobs['fieldDec']-dfobs['AstDec(deg)'], s=0.1, color='darkorchid')
ax.set_ylabel('distance from field centre RA (deg)')
ax.set_xlabel('distance from field centre Dec (deg)')

As can be seen, all of these observations lie on a circle of radius 2.3 degrees. We can also plot this as a histogram.

In [ ]:
object_sep = get_separation(dfobs['AstRA(deg)'].values,
                            dfobs['AstDec(deg)'].values, 
                            dfobs['fieldRA'].values,
                            dfobs['fieldDec'].values)


In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))
_ = ax.hist(object_sep, 100, color='plum')
ax.axvline(2.1, color='black', linestyle='--')
ax.set_xlabel('distance from field centre (deg)')

Now we apply the circle footprint filter with a radius of 1.5 degrees.

In [ ]:
obs_new = PPCircleFootprint(dfobs, 1.5)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))
ax.scatter(obs_new ['fieldRA']-obs_new ['AstRA(deg)'],obs_new ['fieldDec']-obs_new ['AstDec(deg)'], s=0.1, color='darkorchid')
ax.set_ylabel('distance from field centre RA (deg)')
ax.set_xlabel('distance from field centre Dec (deg)')

In [ ]:
object_sep_new = get_separation(obs_new['AstRA(deg)'].values,
                            obs_new['AstDec(deg)'].values, 
                            obs_new['fieldRA'].values,
                            obs_new['fieldDec'].values)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))
_ = ax.hist(object_sep_new, 100, color='plum')
ax.axvline(1.5, color='black', linestyle='--')
ax.set_xlabel('distance from field centre (deg)')